In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
class identity:
    def __init__(self,religion, language):
        '''Creates an identity object consisting of a language parameter between 0 and 1, as well as a categorical value of religion'''
        
        if religion in ['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges' ]:
            self.religion= religion
        else:
            self.religion= 'Sonstiges'
        try:
            if language>=0 and language<=1:
                self.language= language
            else:
                self.language= np.NAN
        except:
            self.language= np.NAN
    
    def get_id(self):
        # Returns the religion and language of the identity object
        return self.religion, self.language

In [ ]:
def comp_ids(id1,id2):
    ''' Compares two identities and returns their similarity index'''
    sim_index=0
    if id1.religion==id2.religion:
        sim_index=0.5
    # When comparing identites the language and religion are weighted equally
    sim_index+=(1-np.abs(id1.language - id2.language))*0.5
    return sim_index

In [ ]:
def Identity_variation (pop_units):
    ''' Calculates the variantion of the identites of a list of population units'''
    ident_list=[]
    religion_list=[]
    language_list=[]
    for po_unit in pop_units:
        ident_list.append(po_unit.identity)
        religion_list.append(po_unit.identity.religion)
        language_list.append(po_unit.identity.language)
    
    var_languages=np.var(language_list)
    #Number of occurences of most frequent religion in group
    occ_max_religion=religion_list.count(max(set(religion_list), key = religion_list.count))
    # Percentage of most frequent religion in group
    perc_max_religion=occ_max_religion/len(religion_list)

    #Adding the 2 metrics together, scaled so that both metrics cover [0,0.5]
    return 1/2*(1- perc_max_religion) + 2*var_languages    

In [ ]:
class pop_unit:
    def __init__(self,pop_size,pop_cooperators,
                 neighbor_units, religion,language, Param):
        '''Creates a population unit object, with its parameters'''
        self.pop_size=pop_size;
        self.pop_cooperators=pop_cooperators;
        self.neighbor_units=[]+ neighbor_units;
        self.pressure=self.calc_pressure();
        self.identity=identity(religion,language);
        
        self.belonging_group=None;
        self.param=Param;
        
    def get_description (self):
        # Prints a description the population units parameters
        print('Identity: ', self.identity.get_id())
        print('Population cooperators: ',self.pop_cooperators)
        print('Population size: ',self.pop_size )
        print('Pressure: ', self.pressure)
        print('Belonging group: ', self.belonging_group)
    
    def add_neighbor(self, neighbor_unit):
        # Adds another population unit to the neighborhood list and recalculates the external pressure
        self.neighbor_units.append(neighbor_unit);
        self.calc_pressure();
        
    def add_group(self, group):
        # Changes the beloning_group of the population unit to the group input
        self.belonging_group= group;
        group.add_member(self);
    
    def iterate(self, repeats):
        ''' The population unit decides/randomly chooses if the cooperator number will increase or decrease by 1 
        and repeats this accordingly to the repeats input'''
        for i in range(repeats):
            if self.pop_cooperators==0:
                # if no cooperator exists, increase it to one cooperator
                self.pop_cooperators=1;
            else:
                #Calculate mu := chances of the cooperator numbers increasing
                mu=self.calc_pressure()*self.pop_cooperators*self.pop_cooperators*(self.pop_size-self.pop_cooperators)
                # Calculate gamma :_ chances of the cooperator numbers decreasing
                gamma=self.param.cost*self.pop_cooperators
                # Increase or decrease the population numbers
                self.pop_cooperators=self.pop_cooperators+np.random.choice([-1,1], p=[gamma/(mu+gamma), mu/(mu+gamma)])

    def calc_pressure(self):
        '''Calculates the pressure a population unit feels from its neighbors'''
        pressure=0;
        for neighbor in self.neighbor_units:
            # If neighbor is in no group/different group, increase pressure, calculated through neighbors size and difference in identity
            if self.belonging_group == None:
                pressure+=neighbor.pop_size/self.pop_size * (1-comp_ids(neighbor.identity, self.identity))
            elif neighbor.belonging_group != self.belonging_group:
                pressure+=neighbor.pop_size/self.pop_size * (1-comp_ids(neighbor.identity, self.identity))
            # If neighbor is in same group, reduce pressure added by 1/4 (1/4 is a model variable to be chosen and can be changed)
            else:
                pressure+=1/4*neighbor.pop_size/self.pop_size * (1-comp_ids(neighbor.identity, self.identity))
        self.pressure=pressure
        return pressure
         

In [ ]:
class pop_group:

    def __init__(self,members):
        '''Creates a population group with a list of members'''
        self.members=members;
        for member in members:
            member.add_group(self);
        
    def in_group(self,member):
        # Checks if member is in group
        return member in self.members;
    
    def add_member(self, member):
        # Adds a member to the group
        if member not in self.members:
            self.members.append(member);
    
    def remove_member(self, member):
        #Removes a member from the group
        if member in self.members:
            self.members.remove(member);

In [ ]:
class Param:
    
    def __init__(self,cost, world, seed):
        # A Parameter Class setting the most important parameters
        self.cost=cost
        self.seed=seed
        
class World:
    ''' Creates the world as a grid of population units, where groups form'''
    def __init__(self, dim_x, dim_y):
        # Creates an empty world of dimension x and y
        self.pop_unit=[];
        self.groups=[];
        self.dim_x=dim_x;
        self.dim_y=dim_y;
        
    def populate_world(self,max_pop_size,min_pop_size, Param):
        ''' Populates the world fully randomly with identity, cooperators and population size(in min and max range) random'''
        np.random.seed(Param.seed)
        units=np.empty([self.dim_x,self.dim_y], object)
        
        #Create the population units for the grid
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                language= np.random.rand()
                pop_size= int(np.floor(np.random.rand()*(max_pop_size - min_pop_size) + min_pop_size))
                pop_cooperators=np.random.choice(range(pop_size),p=np.ones(pop_size)/pop_size)
                units[i,j]= pop_unit(pop_size,pop_cooperators,[], religion,language, Param)
        
        # Adding the neighbors in the grid
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if i-1 >=0:
                    units[i,j].add_neighbor(units[i-1,j])
                if j-1 >=0:
                    units[i,j].add_neighbor(units[i,j-1])
                if i+1 <self.dim_x:
                    units[i,j].add_neighbor(units[i+1,j])
                if j+1< self.dim_y:
                    units[i,j].add_neighbor(units[i,j+1])
        
        self.pop_unit=units;
        
    def populate_world_semi_random(self,pop_size,pop_cooperators, Param):
        ''' Populates the world with randomly selecting the identities of units, but with fixed cooperators and population size numbers'''
        np.random.seed(Param.seed)
        units=np.empty([self.dim_x,self.dim_y], object)
        
        #Create the population units for the grid
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                language= np.random.rand()
                units[i,j]= pop_unit(pop_size,pop_cooperators,[], religion,language, Param)
        
        # Adding the neighbors in the grid
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if i-1 >=0:
                    units[i,j].add_neighbor(units[i-1,j])
                if j-1 >=0:
                    units[i,j].add_neighbor(units[i,j-1])
                if i+1 <self.dim_x:
                    units[i,j].add_neighbor(units[i+1,j])
                if j+1< self.dim_y:
                    units[i,j].add_neighbor(units[i,j+1])
        
        self.pop_unit=units;
        
    def populate_world_semi_random_coop(self,pop_size,religion, Param):
        ''' Populates the world with randomly selecting the language and cooperator numbers of the pop_units, but with fixed religion and population size numbers'''
        np.random.seed(Param.seed)
        units=np.empty([self.dim_x,self.dim_y], object)
        
        #Create the population units for the grid
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                pop_cooperators=np.random.choice(range(pop_size),p=np.ones(pop_size)/pop_size)
                language= np.random.rand()
                units[i,j]= pop_unit(pop_size,pop_cooperators,[], religion,language, Param)
        
        # Adding the neighbors in the grid
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if i-1 >=0:
                    units[i,j].add_neighbor(units[i-1,j])
                if j-1 >=0:
                    units[i,j].add_neighbor(units[i,j-1])
                if i+1 <self.dim_x:
                    units[i,j].add_neighbor(units[i+1,j])
                if j+1< self.dim_y:
                    units[i,j].add_neighbor(units[i,j+1])
        
        self.pop_unit=units;
        
    def populate_world_split(self,pop_size,pop_cooperators_1,pop_cooperators_2,group_1_religion,group_2_religion, Param, languages_random=True, religions_random=False, groups_exist=True):
        ''' Populates the world split into two large groups (split by a diagonal).
        Inputs:
        - pop_size: The population size of all population units
        - pop_cooperators_1: The number of cooperators for group 1
        - pop_cooperators_2: The number of cooperators for group 2
        - group_1_religion: The religion of group 1
        - group_2_religion: The religion of group 2
        - Param: the parameter object for the world
        - languages_random: boolean, if languages should be chosen completely randomly, or in [0,0.5] and [0.5,1] for the groups respectively
        - religions_random: boolean, if religion should be chosen randomly or the parmeters group_1_religion and group_2_religion be used
        - groups_exist: boolean if the groups should actually exist, or only the population units should be created according to the above specification (split by where the groups would be)    
        '''   
        
        np.random.seed(Param.seed)
        units=np.empty([self.dim_x,self.dim_y], object)
        
        #Add the 2 split groups
        if groups_exist:
            self.groups.append(pop_group([]))
            self.groups.append(pop_group([]))
            group_1= self.groups[0]
            group_2= self.groups[1]

        for i in range(self.dim_x):
            for j in range(self.dim_y):
                # Figure out in which coordinate, which population unit should be added (according to the groups)
                if i<=j+int(np.floor((self.dim_x-self.dim_y)/2)):
                    
                    # Figure out language and religion to add
                    if languages_random: language=np.random.rand()
                    else: language=np.random.rand()*0.5
                    if religions_random:
                        religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                    else: religion =group_1_religion
                    
                    # Create Unit
                    units[i,j]= pop_unit(pop_size,pop_cooperators_1,[], religion,language, Param)
                    
                    # Add to Group
                    if groups_exist: units[i,j].add_group(group_1)
                
                elif i+int(np.floor((self.dim_y-self.dim_x)/2))>=j:
                    
                    # Figure out language and religion to add
                    if languages_random: language=np.random.rand()
                    else: language=np.random.rand()*0.5 +0.5
                    if religions_random:
                        religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                    else: religion =group_2_religion
                    
                    # Create Unit
                    units[i,j]= pop_unit(pop_size,pop_cooperators_2,[], religion,language, Param)
                    
                    # Add to Group
                    if groups_exist:units[i,j].add_group(group_2)

        # Make the diagonal of the grid alternate between group 1 and 2
        #Choose if a vertical or a horizontal diagonal would be longer, and populate that diagonal
        if self.dim_x<=self.dim_y:
            for i in range(self.dim_x):
                # Alternate bewwern group 1 and 2
                if i%2==0:
                    #Choose which language and religion to use
                    if languages_random: language=np.random.rand()
                    else: language=np.random.rand()*0.5
                    if religions_random:
                        religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                    else: religion =group_1_religion
                    # Create unit
                    units[i,i+int(np.floor((self.dim_y-self.dim_x)/2))]= pop_unit(pop_size,pop_cooperators_1,[], religion,language, Param)
                    # Add group
                    if groups_exist:units[i,i+int(np.floor((self.dim_y-self.dim_x)/2))].add_group(group_1)
                else:
                    #Choose which language and religion to use
                    if languages_random: language=np.random.rand()
                    else: language=np.random.rand()*0.5+0.5
                    if religions_random:
                        religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                    else: religion =group_2_religion
                    # Create unit
                    units[i,i+int(np.floor((self.dim_y-self.dim_x)/2))]= pop_unit(pop_size,pop_cooperators_2,[], religion,language, Param)
                    # Add group
                    if groups_exist:units[i,i+int(np.floor((self.dim_y-self.dim_x)/2))].add_group(group_2)
        else:
            for i in range(self.dim_y):
                # Alternate bewwern group 1 and 2
                if i%2==0:
                    #Choose which language and religion to use
                    if languages_random: language=np.random.rand()
                    else: language=np.random.rand()*0.5
                    if religions_random:
                        religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                    else: religion =group_1_religion
                    # Create unit
                    units[i+int(np.floor((self.dim_x-self.dim_y)/2)),i]= pop_unit(pop_size,pop_cooperators_1,[], religion,language, Param)
                    # Add group
                    if groups_exist:units[i+int(np.floor((self.dim_x-self.dim_y)/2)),i].add_group(group_1)
                else:
                    #Choose which language and religion to use
                    if languages_random: language=np.random.rand()
                    else: language=np.random.rand()*0.5 +0.5
                    if religions_random:
                        religion=np.random.choice(['Judentum', 'Islam', 'Christentum', 'Hinduismus', 'Buddhismus', 'keine', 'Sonstiges'], p=np.ones(7)/7)
                    else: religion =group_2_religion
                    # Create unit  
                    units[i+int(np.floor((self.dim_x-self.dim_y)/2)),i]= pop_unit(pop_size,pop_cooperators_2,[], religion,language, Param)
                    # Add group
                    if groups_exist:units[i+int(np.floor((self.dim_x-self.dim_y)/2)),i].add_group(group_2)
        
        # Connect neighbors
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if i-1 >=0:
                    units[i,j].add_neighbor(units[i-1,j])
                if j-1 >=0:
                    units[i,j].add_neighbor(units[i,j-1])
                if i+1 <self.dim_x:
                    units[i,j].add_neighbor(units[i+1,j])
                if j+1< self.dim_y:
                    units[i,j].add_neighbor(units[i,j+1])
        
        self.pop_unit=units;
        
    def check_merge(self,pop_unit_1, pop_unit_2):
        '''Check if two population units would want to merge, and returns a metric measuring the willingness to merge'''
        #Calculates the average of their opinion numbers to cooperate from their respective populations
        opinions=0.5*(pop_unit_1.pop_cooperators/pop_unit_1.pop_size + pop_unit_2.pop_cooperators/pop_unit_2.pop_size)
        # Calculate the similarity of their identities
        similarity= comp_ids(pop_unit_1.identity, pop_unit_2.identity)
        # Return the weighted metric (0.7 and 0.3 are model variables to be chosen and can be changed)
        return 0.7*opinions+0.3*similarity
    
    def group_merging (self):
        ''' Check if and then merge all population units'''
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                # Only check, if population units does not have a group yet
                if self.pop_unit[i,j].belonging_group == None:
                    # Collect all merging values of the neighbors
                    merge_values_neighbors=[]
                    for neighbor in self.pop_unit[i,j].neighbor_units:
                        merge_values_neighbors.append(self.check_merge(self.pop_unit[i,j], neighbor))
                    # if the maximum value exceeds 0.5, merge with the maximum value neighbor
                    if np.max(merge_values_neighbors)> 0.5:
                        to_merge=self.pop_unit[i,j].neighbor_units[np.argmax(merge_values_neighbors)]
                        # if neighbor alread is already a part of a group, the population unit joins that group
                        if to_merge.belonging_group != None:
                            self.pop_unit[i,j].add_group(to_merge.belonging_group);
                        # Otherwise a new group is created
                        else:  
                            self.groups.append(pop_group([self.pop_unit[i,j],to_merge]));
                            
    def group_dissolving_split (self, group):
        '''Checks if the group has population units, which want to dissolve, and those units leave the group'''
        
        if group== None: return
        # Calculate the identity variation of all group members
        id_var=Identity_variation(group.members)
        for pop_unit in group.members:
            # Calculate the disolving threshhold (0.7 and 0.3 are model variables to be chosen and can be changed)
            disolv_threshhold=0.7*(1-pop_unit.pop_cooperators/pop_unit.pop_size) + 0.3*id_var
            # If threshhold 50% or above, then the population unit leaves the group
            if disolv_threshhold>=0.5:
                pop_unit.belonging_group=None
                group.remove_member(pop_unit)
                # If group then only has one member, destroy the group
                if len(group.members)==1:
                    self.groups[self.groups.index(group)]=None
                    group.members[0].belonging_group=None
                    group.remove_member(group.members[0])
                    
    def group_dissolving_complete (self, group):
        '''Checks if 50% or more population units of the group want to dissolve and dissolves it in that case'''
        
        if group== None: return
        # Calculate the identity variation of all group members
        id_var=Identity_variation(group.members)
        disolv_threshholds=[]
        for pop_unit in group.members:
            # Calculate the disolving threshhold (0.7 and 0.3 are model variables to be chosen and can be changed)
            disolv_threshholds.append(0.7*(1-pop_unit.pop_cooperators/pop_unit.pop_size) + 0.3*id_var)
        # If more than 50% of the group members wan to leave the group, dissolve the group
        if len([1 for tresh in disolv_threshholds if tresh>0.5])/len(disolv_threshholds)>=0.5:
            for pop_unit in group.members:
                pop_unit.belonging_group=None
            group.members=[]
            self.groups[self.groups.index(group)]=None
            

    def all_group_dissolving(self, dissolving_type):
        # Check for dissolving of the group, depending on the dissolving type
        if dissolving_type== 'split':
            for group in self.groups:
                self.group_dissolving_split(group)
        elif dissolving_type== 'complete':
            for group in self.groups:
                self.group_dissolving_complete(group)
    
    def iterate_all (self, number):
        # Iterate all population units by number
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                self.pop_unit[i,j].iterate(number)
    
    def visualize_groups (self):
        # Visualize the current groups, that all the population units in the world belong to
        group_overview=np.zeros([self.dim_x,self.dim_y], object)
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if self.pop_unit[i,j].belonging_group !=None:
                    group_overview[i,j]=self.groups.index(self.pop_unit[i,j].belonging_group)+1
        return group_overview
    
    def visualize_feature (self,feature):
        # Visualize a certain feature of the population units in the world
        group_overview=np.empty([self.dim_x,self.dim_y], object)
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if getattr(self.pop_unit[i,j],feature) !=None:
                    group_overview[i,j]=getattr(self.pop_unit[i,j],feature)
        return group_overview
    
    def visualize_identity (self):
        # Visualize the identity metrics of the population units in the world
        group_overview=np.empty([self.dim_x,self.dim_y], object)
        for i in range(self.dim_x):
            for j in range(self.dim_y):
                if self.pop_unit[i,j].identity !=None:
                    group_overview[i,j]=self.pop_unit[i,j].identity.get_id()
        return group_overview
    
    

# Running simulation functions

In [ ]:
def run_simulation (world,loops, growth_period, dissolving_type, print_ = False):
    ''' Run a simulation of the world, where we for 'loops' times:
    - iterate all population units for the growth period
    - check if groups dissolve
    - check if populations emerge'''
    for i in range(loops):
        if print_: print('loop number: ', i)
        world.iterate_all(growth_period)
        if print_: print("Dissolved:")
        world.all_group_dissolving(dissolving_type)
        if print_: print(world.visualize_groups())
        if print_: print("Merged:")
        world.group_merging()
        if print_: print(world.visualize_groups())

In [ ]:
def run_simulation_evaluation_2_nations_single (worldsize, pop_size, cooperators_1,cooperators_2, religions,dissolving_type, iterations, growth_period, languages_random=True, religions_random=False, print_=True):
    ''' 
    Simulates the 2 nation starting conditions and records the changes over time in:
    - group size average
    - number of groups exisiting
    - % of population units in groups
    
    Input:
    - worldsize: list of the x and y grid size for the world
    - pop_size: Population size of the population units in the world
    - cooperators_1: Number of cooperators for group 1
    - cooperators_2: Number of cooperators for group 2
    - religions: list of the religions for group 1 and 2
    - dissolving_type: dissolving type to consider for the simulation
    - iterations: how often should the simulation run
    - growth_period: the growth period all population units iterate over in one step of the simulation
    - languages_random: boolean, if languages should be chosen completely randomly, or in [0,0.5] and [0.5,1] for the groups respectively
    - religions_random: boolean, if religion should be chosen randomly or the parmeters group_1_religion and group_2_religion be used
    - print_: boolean if outputs should be printed
   
   Returns:
    - group_size_avg: Average group size over time
    - groups_existing: Number of groups existing over time
    - in_groups_perc: Percentage of population units in groups   
    '''
    time_till_devolving=[]
    group_size_avg=[]
    groups_existing=[]
    in_groups_perc=[]
    for i in range(iterations):
        world= World (worldsize[0],worldsize[1])
        # Cost chosen to match potential maximum 
        world.populate_world_split(pop_size,cooperators_1, cooperators_2, religions[0],religions[1], Param(0.9*pop_size*pop_size/4,20+i),languages_random, religions_random)
        
        for l in range(10):
            visualization=world.visualize_groups()
            run_simulation(world, 10,growth_period,dissolving_type)
            group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
            groups_existing.append(len([group for group in world.groups if group!=None]))
            in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
            if print_: print('group_size_avg ',l,' period:',np.mean([len(group.members) for group in world.groups if group!=None]))
            if print_: print('groups_existing ',l,' period:',len([group for group in world.groups if group!=None]))
            if print_: print('in groups % ',l,' period:',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
            print(world.visualize_groups())
    return group_size_avg, groups_existing, in_groups_perc

In [ ]:
def run_simulation_evaluation_2_nations (worldsize, pop_size, cooperators_1,cooperators_2, religions,dissolving_type, iterations, growth_period, languages_random=True, religions_random=False):
    ''' 
    Simulates the 2 nation starting conditions and records the time till one of the nations first dissolves
    
    Input:
    - worldsize: list of the x and y grid size for the world
    - pop_size: Population size of the population units in the world
    - cooperators_1: Number of cooperators for group 1
    - cooperators_2: Number of cooperators for group 2
    - religions: list of the religions for group 1 and 2
    - dissolving_type: dissolving type to consider for the simulation
    - iterations: how often should the simulation run
    - growth_period: the growth period all population units iterate over in one step of the simulation
    - languages_random: boolean, if languages should be chosen completely randomly, or in [0,0.5] and [0.5,1] for the groups respectively
    - religions_random: boolean, if religion should be chosen randomly or the parmeters group_1_religion and group_2_religion be used
   
   Returns:
    - time_till_devolving: List of all times till one nation dissolved
    '''
    time_till_devolving=[]
    for i in range(iterations):
        world= World (worldsize[0],worldsize[1])
        # Cost chosen to match potential maximum 
        world.populate_world_split(pop_size,cooperators_1, cooperators_2, religions[0],religions[1], Param(0.9*pop_size*pop_size/4,20+i),languages_random, religions_random)
        
        group_1=world.groups[0]
        group_2=world.groups[1]
        for l in range(1000):
            group_1_members=len(group_1.members)
            group_2_members=len(group_2.members)
            run_simulation(world, 1,growth_period,dissolving_type)
            if len(group_1.members)<group_1_members or len(group_2.members)<group_2_members:
                time_till_devolving.append(l)
                break
    return time_till_devolving

In [ ]:
def run_simulation_evaluation_semi_random (worldsize, pop_size, cooperators,dissolving_type, iterations, growth_period,relig_rand=True, religion='Sonstiges'):
    ''' 
    Simulates the semi random starting conditions and records the following in the final conditions:
    - group size average
    - number of groups exisiting
    - % of population units in groups
    
    Input:
    - worldsize: list of the x and y grid size for the world
    - pop_size: Population size of the population units in the world
    - cooperators: Number of cooperators
    - dissolving_type: dissolving type to consider for the simulation
    - iterations: how often should the simulation run
    - growth_period: the growth period all population units iterate over in one step of the simulation
    - relig_rand: boolean, if religion should be chosen randomly or the parmeters religion should be used
    - religion: religion to be used if relig_rand is false
   
   Returns:
    - group_size_avg: List of average group size in the end states per simulation
    - groups_existing:  List of number of groups existing in the end states per simulation
    - in_groups_perc: List of percentage of population units in groups in the end states per simulation
    '''
    group_size_avg=[]
    groups_existing=[]
    in_groups_perc=[]
    for i in range(iterations):
        print('iteration ',i)
        world= World (worldsize[0],worldsize[1])
        if relig_rand: world.populate_world_semi_random(pop_size,cooperators, Param(0.9*pop_size*pop_size/4,20+i))
        else: 
            world.populate_world_semi_random_coop(pop_size,religion, Param(0.9*pop_size*pop_size/4,20+i))
        
        run_simulation(world, 100,growth_period,dissolving_type)
        added=False
        for l in range(10):
            visualization=world.visualize_groups()
            run_simulation(world, 10,growth_period,dissolving_type)
            if (visualization==world.visualize_groups()).all():
                group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
                groups_existing.append(len([group for group in world.groups if group!=None]))
                in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
                print('group_size_avg: ',np.mean([len(group.members) for group in world.groups if group!=None]))
                print('groups_existing: ',len([group for group in world.groups if group!=None]))
                print('in groups %: ',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
                added=True
                break
        if added==False:
            group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
            groups_existing.append(len([group for group in world.groups if group!=None]))
            in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
            print('group_size_avg: ',np.mean([len(group.members) for group in world.groups if group!=None]))
            print('groups_existing: ',len([group for group in world.groups if group!=None]))
            print('in groups %: ',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
                
    return group_size_avg, groups_existing, in_groups_perc

def run_simulation_evaluation_random (worldsize, max_pop_size,min_pop_size,dissolving_type, iterations, growth_period):
    ''' 
    Simulates the random starting conditions and records the following in the final conditions:
    - group size average
    - number of groups exisiting
    - % of population units in groups
    
    Input:
    - worldsize: list of the x and y grid size for the world
    - max_pop_size: Max population size of the population units in the world
    - min_pop_size: Min population size of the population units in the world
    - dissolving_type: dissolving type to consider for the simulation
    - iterations: how often should the simulation run
    - growth_period: the growth period all population units iterate over in one step of the simulation
   
   Returns:
    - group_size_avg: List of average group size in the end states per simulation
    - groups_existing:  List of number of groups existing in the end states per simulation
    - in_groups_perc: List of percentage of population units in groups in the end states per simulation
    '''
    group_size_avg=[]
    groups_existing=[]
    in_groups_perc=[]
    for i in range(iterations):
        print('iteration ',i)
        world= World (worldsize[0],worldsize[1])
        world.populate_world(max_pop_size,min_pop_size, Param(0.9*max_pop_size*max_pop_size/4,20+i))
        run_simulation(world, 100,growth_period,dissolving_type)
        added=False
        for l in range(10):
            visualization=world.visualize_groups()
            run_simulation(world, 10,growth_period,dissolving_type)
            if (visualization==world.visualize_groups()).all():
                group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
                groups_existing.append(len([group for group in world.groups if group!=None]))
                in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
                print('group_size_avg: ',np.mean([len(group.members) for group in world.groups if group!=None]))
                print('groups_existing: ',len([group for group in world.groups if group!=None]))
                print('in groups %: ',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
                added=True
                break
        if added==False:
            group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
            groups_existing.append(len([group for group in world.groups if group!=None]))
            in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
            print('group_size_avg: ',np.mean([len(group.members) for group in world.groups if group!=None]))
            print('groups_existing: ',len([group for group in world.groups if group!=None]))
            print('in groups %: ',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
                
    return group_size_avg, groups_existing, in_groups_perc

In [ ]:
def run_simulation_evaluation_random_time (worldsize, max_pop_size,min_pop_size,dissolving_type, growth_period, iterations, seed=20, print_=False):
    ''' 
    Simulates the random starting conditions and records the changes over time in:
    - group size average
    - number of groups exisiting
    - % of population units in groups
    
    Input:
    - worldsize: list of the x and y grid size for the world
    - max_pop_size: Max population size of the population units in the world
    - min_pop_size: Min population size of the population units in the world
    - dissolving_type: dissolving type to consider for the simulation
    - growth_period: the growth period all population units iterate over in one step of the simulation
    - iterations: how often should the simulation run
    - seed: The random seed number for the first simulation
    - print_: boolean if outputs should be printed
   
   Returns:
    - group_size_avg: Average group size over time
    - groups_existing: Number of groups existing over time
    - in_groups_perc: Percentage of population units in groups over time
    '''
    group_size_avg=[]
    groups_existing=[]
    in_groups_perc=[]
    world= World (worldsize[0],worldsize[1])
    world.populate_world(max_pop_size,min_pop_size, Param(0.9*max_pop_size*max_pop_size/4,seed))
    for l in range(iterations):
        visualization=world.visualize_groups()
        run_simulation(world, 1,growth_period,dissolving_type)
        group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
        groups_existing.append(len([group for group in world.groups if group!=None]))
        in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
        if print_: print('group_size_avg ',l,' period:',np.mean([len(group.members) for group in world.groups if group!=None]))
        if print_: print('groups_existing ',l,' period:',len([group for group in world.groups if group!=None]))
        if print_: print('in groups % ',l,' period:',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
    print(world.visualize_groups())
    return group_size_avg, groups_existing, in_groups_perc

def run_simulation_evaluation_semi_random_time (worldsize, pop_size,cooperators,dissolving_type, iterations, growth_period, seed=20, print_=False):
    ''' 
    Simulates the semi random starting conditions and records the changes over time in:
    - group size average
    - number of groups exisiting
    - % of population units in groups
    
    Input:
    - worldsize: list of the x and y grid size for the world
    - pop_size: Population size of the population units in the world
    - cooperators: Number of cooperators
    - dissolving_type: dissolving type to consider for the simulation
    - growth_period: the growth period all population units iterate over in one step of the simulation
    - iterations: how often should the simulation run
    - seed: The random seed number for the first simulation
    - print_: boolean if outputs should be printed
   
   Returns:
    - group_size_avg: Average group size over time
    - groups_existing: Number of groups existing over time
    - in_groups_perc: Percentage of population units in groups over time
    '''
    group_size_avg=[]
    groups_existing=[]
    in_groups_perc=[]
    world= World (worldsize[0],worldsize[1])
    world.populate_world_semi_random(pop_size,cooperators, Param(0.9*pop_size*pop_size/4,seed))
    for l in range(iterations):
        visualization=world.visualize_groups()
        run_simulation(world, 1,growth_period,dissolving_type)
        group_size_avg.append(np.mean([len(group.members) for group in world.groups if group!=None]))
        groups_existing.append(len([group for group in world.groups if group!=None]))
        in_groups_perc.append(np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]))
        if print_: print('group_size_avg ',l,' period:',np.mean([len(group.members) for group in world.groups if group!=None]))
        if print_: print('groups_existing ',l,' period:',len([group for group in world.groups if group!=None]))
        if print_: print('in groups % ',l,' period:',np.sum([len(group.members) for group in world.groups if group!=None])/(worldsize[0]*worldsize[1]) )
    print(world.visualize_groups())
    return group_size_avg, groups_existing, in_groups_perc

# Running simulations

## 2 Nations

In [ ]:
# Test single simulation of 2 nation case
run_simulation_evaluation_2_nations_single ([10,10], 200, 100,150, ['Judentum', 'Sonstiges'],'complete',1,20, True, False)

In [ ]:
# Running multiple test cases for the 2 nations starting conditions
output_1=run_simulation_evaluation_2_nations ([10,10], 200, 50,50, ['Judentum', 'Judentum'],'split',100,20)
output_2=run_simulation_evaluation_2_nations ([10,10], 200, 50,50, ['Judentum', 'Judentum'],'complete',100,20)
output_3=run_simulation_evaluation_2_nations ([10,10], 200, 100,100, ['Judentum', 'Judentum'],'split',100,20)
output_4=run_simulation_evaluation_2_nations ([10,10], 200, 100, 100, ['Judentum', 'Judentum'],'complete',100,20)
output_5=run_simulation_evaluation_2_nations ([10,10], 200, 150, 150, ['Judentum', 'Judentum'],'split',100,20)
output_6=run_simulation_evaluation_2_nations ([10,10], 200, 150, 150, ['Judentum', 'Judentum'],'complete',100,20)
output_7=run_simulation_evaluation_2_nations ([10,10], 200, 50, 50, ['Judentum', 'Islam'],'split',100,20)
output_8=run_simulation_evaluation_2_nations ([10,10], 200, 50, 50, ['Judentum', 'Islam'],'complete',100,20)
output_9=run_simulation_evaluation_2_nations ([10,10], 200, 100, 100, ['Judentum', 'Islam'],'split',100,20)
output_10=run_simulation_evaluation_2_nations ([10,10], 200, 100, 100, ['Judentum', 'Islam'],'complete',100,20)
output_11=run_simulation_evaluation_2_nations ([10,10], 200, 150, 150, ['Judentum', 'Islam'],'split',100,20)
output_12=run_simulation_evaluation_2_nations ([10,10], 200, 150, 150, ['Judentum', 'Islam'],'complete',100,20)

results_2_nations=[]
for i in [output_1,output_2,output_3,output_4,output_5,output_6,output_7,output_8,output_9,output_10,output_11,output_12]:
    results_2_nations.append(np.mean(i))

In [ ]:
# Gathering all results in a data frame
cooperators_2_nations=[50,50,100,100,150,150]+[50,50,100,100,150,150]
religion_1=['Judentum']
for i in range(11):
    religion_1=religion_1 +['Judentum']
religion_2=['Judentum']
for i in range(5):
    religion_2=religion_2 +['Judentum']
for i in range(6):
    religion_2=religion_2 +['Islam']
dissolv_type=['separate', 'dissolution']
for i in range(5):
    dissolv_type=dissolv_type +['separate', 'dissolution']

df_2=pd.DataFrame(results_2_nations, columns=['time till first dissolve'])
df_2['cooperators']=cooperators_2_nations
df_2['dissolving-type']=dissolv_type
df_2['religion 1']=religion_1
df_2['religion 2']=religion_2
df_2[['cooperators','dissolving-type','religion 1','religion 2','time till first dissolve']].sort_values('cooperators').reset_index(drop=True)

## Semi Random

In [ ]:
group_size_avg_semi,groups_existing_semi=run_simulation_evaluation_semi_random_time([10,10], 200, 50, 'split', 20, 200, seed=20)

In [ ]:
# Cooperators fixed, religion random
pop_size=200
cooperators=[50,100,150]
dissolving_type=['seperate', 'complete']
growth_period= 20
iterations=10
results_semi=[]
for coop in cooperators:
    for dissolv in dissolving_type:
        print('cooperators: ', coop,' population size:',pop_size, ' dissolving type: ', dissolv)
        group_size_avg,groups_existing,in_groups_perc=run_simulation_evaluation_semi_random([10,10], pop_size, coop, dissolv, iterations, growth_period)
        print('averge group size: ', np.mean(group_size_avg))
        print('groups_existing: ', np.mean(groups_existing))
        print('in_groups_perc: ', np.mean(in_groups_perc))
        results_semi.append([np.mean(group_size_avg),np.mean(groups_existing),np.mean(in_groups_perc)])

In [ ]:
# Cooperators fixed, Religion random
df_semi_random=pd.DataFrame(results_semi, columns=['average group size','number of groups existing','% in groups'])
df_semi_random['cooperators']=[50,50,100,100,150,150]
df_semi_random['dissolving-type']=['seperate', 'dissolution']*3
df_semi_random[['cooperators','dissolving-type','average group size','number of groups existing','% in groups']]

In [ ]:
# Cooperators random, Religion fixed
relig_rand_=False
religion_='Sonstiges'
pop_size=200
dissolving_type=['split', 'complete']
growth_period= 20
iterations=10
results_semi_2=[]

for dissolv in dissolving_type:
    print('population size:',pop_size, ' dissolving type: ', dissolv)
    group_size_avg,groups_existing,in_groups_perc=run_simulation_evaluation_semi_random([10,10], pop_size, 0, dissolv, iterations, growth_period,relig_rand_,religion_)
    print('averge group size: ', np.mean(group_size_avg))
    print('groups_existing: ', np.mean(groups_existing))
    print('in_groups_perc: ', np.mean(in_groups_perc))
    results_semi_2.append([np.nanmean(group_size_avg),np.mean(groups_existing),np.mean(in_groups_perc)])

In [ ]:
# Cooperators random, Religion fixed
df_semi_random_2=pd.DataFrame(results_semi_2, columns=['average group size','number of groups existing','% in groups'])
df_semi_random_2['religion']=religion_
df_semi_random_2['dissolving-type']=['seperate', 'dissolution']
df_semi_random_2[['religion','dissolving-type','average group size','number of groups existing','% in groups']]

## Random

In [ ]:
max_pop=200
min_pop=[1,50,100,150,200]
seed=[20,21,22,23]
growth_period= 20
periods=200

In [ ]:
seed_i=0
min_pop_i=2
group_size_avg,groups_existing,in_groups_perc=run_simulation_evaluation_random_time([10,10], max_pop, min_pop[min_pop_i], 'complete', growth_period, periods, seed=seed[seed_i])
ax=pd.DataFrame(group_size_avg).plot()
plt.legend(['average group size'])
ax.set_xlabel("periods")

ax=pd.DataFrame(groups_existing).plot()
plt.legend(['groups existing'])
ax.set_xlabel("periods")

ax=pd.DataFrame(in_groups_perc).plot()
plt.legend(['% of pop. units in groups'])
ax.set_xlabel("periods")

### Metrics Evaluated in end states

In [ ]:
group_size_avg,groups_existing,in_groups_perc=run_simulation_evaluation_random([10,10], 200, 100,'split', 10, 20)

In [ ]:
max_pop=200
min_pop=[1,50,100,150,200]
dissolving_type=['split', 'complete']
growth_period= 20
iterations=10
results=[]
for min_p in min_pop:
    for dissolv in dissolving_type:
        print('minimum population: ', min_p,' max population:',max_pop, ' dissolving type: ', dissolv)
        group_size_avg,groups_existing,in_groups_perc=run_simulation_evaluation_random([10,10], max_pop, min_p,dissolv, iterations, growth_period)
        print('averge group size: ', np.mean(group_size_avg))
        print('groups_existing: ', np.mean(groups_existing))
        print('in_groups_perc: ', np.mean(in_groups_perc))
        results.append([np.nanmean(group_size_avg),np.mean(groups_existing),np.mean(in_groups_perc)])

In [ ]:
df_random=pd.DataFrame(results, columns=['average group size','number of groups existing','% in groups'])
df_random['min. pop. size']=[1,1,50,50,100,100,150,150,200,200]
dissolving_types=['seperate', 'dissolution']
df_random['dissolving-type']=dissolving_types*5
df_random[['min. pop. size','dissolving-type','average group size','number of groups existing','% in groups']]